In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import tensorflow_hub as hub
import tensorflow as tf

In [2]:
data = "C:/Users/Christian Musni/Downloads/dataset"

In [3]:
class_names = os.listdir(data)
class_paths = [os.path.join(data, cls) for cls in class_names]

In [4]:
num_classes = len(class_names)

In [5]:
file_paths = []
labels = []

In [6]:
for idx, class_path in enumerate(class_paths):
    files = os.listdir(class_path)
    file_paths.extend([os.path.join(class_path, file) for file in files])
    labels.extend([idx] * len(files))

In [7]:
# Split the data into train and test sets
train_files, test_files, train_labels, test_labels = train_test_split(file_paths, labels, test_size=0.2, random_state=42)

In [8]:
# Define data augmentation for both training and validation data
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% validation split
)

# Set the target image size
target_size = (224, 224)  # Adjust based on your EfficientNetV2 input size

# Create train and validation generators
train_generator = datagen.flow_from_directory(
    data,
    target_size=target_size,
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data,
    target_size=target_size,
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 807 images belonging to 10 classes.
Found 199 images belonging to 10 classes.


In [9]:
# Define the EfficientNetV2 model from TensorFlow Hub
efficientnetv2_url = 'https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_l/feature_vector/2'
efficientnetv2 = hub.KerasLayer(efficientnetv2_url)

# Create a new model on top of EfficientNetV2
model = tf.keras.Sequential([
    efficientnetv2,
    # tf.keras.layers.Dense(128, activation = 'relu'),
    # tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
model.fit(train_generator, epochs=3, validation_data=validation_generator, verbose=1)

Epoch 1/3
26/26 [==============================] - 58s 895ms/step - loss: 1.9259 - accuracy: 0.4002 - val_loss: 1.5982 - val_accuracy: 0.5779
Epoch 2/3
26/26 [==============================] - 20s 753ms/step - loss: 1.3108 - accuracy: 0.6530 - val_loss: 1.2895 - val_accuracy: 0.5980
Epoch 3/3
26/26 [==============================] - 30s 1s/step - loss: 1.0185 - accuracy: 0.7175 - val_loss: 1.1684 - val_accuracy: 0.5829


In [13]:
model.save('C:/Users/Christian Musni/Downloads/dataset/rice_crop_stress_classifier_effnetv2_large.h5')

In [15]:
from tensorflow.keras.preprocessing import image
# Load an image for testing
img_path = 'C:/Users/Christian Musni/Downloads/new/test.jpg'
img = image.load_img(img_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

# Make predictions
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions)

1/1 [==============================] - 3s 3s/step


In [16]:
from tensorflow.keras.preprocessing import image
import numpy as np

def load_and_preprocess_image(image_path, target_size):
    img = image.load_img(image_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

In [20]:
# Replace 'image_path' with the path to your test image
image_path = 'C:/Users/Christian Musni/Downloads/new/test1.jpg'

# Load and preprocess the image
input_image = load_and_preprocess_image(image_path, target_size=(300, 300))  # Use the appropriate target size

# Make predictions
predictions = model.predict(input_image)

1/1 [==============================] - 0s 261ms/step


In [21]:
import numpy as np

# Assuming `class_names` is a list of your class labels
class_names = ['blast', 'bacterial_leaf_blight', 'tungro', 'sheath_blight', 'brown_plant_hopper',
               'green_leaf_hopper', 'yellow_stem_borer', 'stem_borer', 'false_smut', 'healthy']

predicted_class = np.argmax(predictions)  # Get the index of the class with the highest probability
predicted_label = class_names[predicted_class]  # Get the label using the index

# Get the probability of the predicted class
probability = predictions[0][predicted_class]  # Assuming predictions is of shape (1, num_classes)

print(f"Predicted class: {predicted_label}")
print(f"Probability: {probability}")

Predicted class: green_leaf_hopper
Probability: 0.9993785619735718
